# Supply chain physics
*This notebook illustrates methods to investigate the physics of a supply chain*
***
Alessandro Tufano 2020

### Import packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt


### Generate empirical demand and production  
We define an yearly sample of production quantity $x$, and demand quantity $d$

In [2]:
number_of_sample = 365 #days
mu_production = 105 #units per day
sigma_production = 1 # units per day

mu_demand = 100 #units per day
sigma_demand = 0.3 # units per day


x = np.random.normal(mu_production,sigma_production,number_of_sample) 
#d = np.random.normal(mu_demand,sigma_demand,number_of_sample) 
d = brownian(x0=mu_demand, n=365, dt=1, delta=sigma_demand, out=None) #demand stochastic process
# represent demand
plt.hist(d,color='orange')
plt.hist(x,color='skyblue')

plt.title('Production and Demand histogram')
plt.xlabel('Daily rate')
plt.ylabel('Frequency')
plt.legend(['Demand','Production'])

x = np.array(x)
d = np.array(d)

plt.figure()
plt.plot(d)
plt.title("Demand curve $d$")
plt.xlabel('Time in days')
plt.ylabel('Numbar of parts')

plt.figure()
plt.plot(x)
plt.title("Production curve $x$")
plt.xlabel('Time in days')
plt.ylabel('Number of parts')

NameError: name 'brownian' is not defined

### Define the inventory function $q$  
The empirical inventory function $q$ is defined as the differende between production and demand, plus the residual inventory.  
$q_t = q_{t-1} + x_t - d_t$

In [ ]:
q = [mu_production] #initial inventory with production mean value
for i in range(0,len(d)):
    inventory_value = q[i] + x[i] - d[i] 
    if inventory_value <0 : 
        inventory_value=0
    q.append(inventory_value)
    
plt.plot(q)
plt.xlabel('days')
plt.ylabel('Inventory quantity $q$')
plt.title('Inventory function $q$')

q = np.array(q)

### Define pull and push forces (the momentum $p=\dot{q}$)  
By using continuous notation we obtain the derivative $\dot{q}=p=x-d$. The derivative of the inventory represents the *momentum* of the supply chain, i.e. the speed a which the inventory values goes up (production), and down (demand). We use the term **productivity** to identify the momentum $p$. The forces changing the value of the productivity are called **movements** $\dot{p}$.

In [ ]:
p1 = [q[i]-q[i-1] for i in range(1,len(q))]
p2 = [x[i]-d[i] for i in range(1,len(d))]
plt.plot(p1)
plt.plot(p2)
plt.xlabel('days')
plt.ylabel('Value')
plt.title('Momentum function $p$')

p=np.array(p)

### Define a linear potential $V(q)$  
we introduce a linear potential to describe the amount of *energy* related with a given quantity of the inventory $q$.

In [ ]:
F0 = 0.1

#eta = 1.2
#lam = mu_demand
#F0=eta*lam

print(F0)

V_q = -F0*q
V_q = V_q[0:-1]

### Define the energy conservation function using the Lagrangianm and the Hamiltonian  
We use the Lagrangian to describe the energy conservation equation.
$L(q,\dot{q}) = H = \frac{1}{2}\dot{q} - V(q)$

In [ ]:
H = (p**2)/2 - F0*q[0:-1]

plt.plot(H)
plt.xlabel('days')
plt.ylabel('value')
plt.title('Function $H$')

### Obtain the inventory $q$, given $H$

In [ ]:
S_q = [H[i-1] + H[i] for i in range(1,len(H))]
plt.plot(S_q)
plt.xlabel('days')
plt.ylabel('value')
plt.title('Function $S[q]$')


#compare with q
plt.plot(q)
plt.xlabel('days')
plt.ylabel('Inventory quantity $q$')
plt.title('Inventory function $q$')

plt.legend(['Model inventory','Empirical inventory'])

# Inventory control

### Define the Brownian process

In [ ]:
from math import sqrt
from scipy.stats import norm
import numpy as np


def brownian(x0, n, dt, delta, out=None):
    """
    Generate an instance of Brownian motion (i.e. the Wiener process):

        X(t) = X(0) + N(0, delta**2 * t; 0, t)

    where N(a,b; t0, t1) is a normally distributed random variable with mean a and
    variance b.  The parameters t0 and t1 make explicit the statistical
    independence of N on different time intervals; that is, if [t0, t1) and
    [t2, t3) are disjoint intervals, then N(a, b; t0, t1) and N(a, b; t2, t3)
    are independent.
    
    Written as an iteration scheme,

        X(t + dt) = X(t) + N(0, delta**2 * dt; t, t+dt)


    If `x0` is an array (or array-like), each value in `x0` is treated as
    an initial condition, and the value returned is a numpy array with one
    more dimension than `x0`.

    Arguments
    ---------
    x0 : float or numpy array (or something that can be converted to a numpy array
         using numpy.asarray(x0)).
        The initial condition(s) (i.e. position(s)) of the Brownian motion.
    n : int
        The number of steps to take.
    dt : float
        The time step.
    delta : float
        delta determines the "speed" of the Brownian motion.  The random variable
        of the position at time t, X(t), has a normal distribution whose mean is
        the position at time t=0 and whose variance is delta**2*t.
    out : numpy array or None
        If `out` is not None, it specifies the array in which to put the
        result.  If `out` is None, a new numpy array is created and returned.

    Returns
    -------
    A numpy array of floats with shape `x0.shape + (n,)`.
    
    Note that the initial value `x0` is not included in the returned array.
    """

    x0 = np.asarray(x0)

    # For each element of x0, generate a sample of n numbers from a
    # normal distribution.
    r = norm.rvs(size=x0.shape + (n,), scale=delta*sqrt(dt))

    # If `out` was not given, create an output array.
    if out is None:
        out = np.empty(r.shape)

    # This computes the Brownian motion by forming the cumulative sum of
    # the random samples. 
    np.cumsum(r, axis=-1, out=out)

    # Add the initial condition.
    out += np.expand_dims(x0, axis=-1)

    return out

### Define the supply chain control model

In [ ]:
# supply chain control model
def supply_chain_control_model(p,beta,eta,F0):
    #p is the productivity function defined as the defivative of q
    #beta is the diffusion coefficient, i.e. the delta of the Brownian process, the std of the demand can be used
    #eta represents the flexibility of the productio. It is the number of days to reach a target inventory
    #F0 is the potential   
    
    Fr_t = brownian(x0=F0, n=365, dt=1, delta=beta, out=None) #demand stochastic process
    p_dot = F0 -eta*p + Fr_t
    return p_dot, Fr_t

In [ ]:
#identify the sensitivity of the inventory control with different values of eta
for eta in [0.1,1,2,7,30]:
    p_dot, Fr_t = supply_chain_control_model(p=p,beta = sigma_demand,eta=eta,F0=F0)
    

    plt.figure()
    plt.plot(Fr_t)
    plt.plot(p)
    plt.plot(p_dot)
    plt.title(f"Inventory control with eta={eta}")
    plt.legend(['Demand','Productivity','Movements $\dot{p}$'])

In [ ]:
p_dot, Fr_t = supply_chain_control_model(p=p,beta = sigma_demand,eta=1,F0=0.9)
p_model = [p_dot[i-1] + p_dot[i] for i in range(1,len(p_dot))]
q_model = [p_model[i-1] + p_model[i] for i in range(1,len(p_model))]


plt.plot(q_model)
plt.plot(p_model)
plt.legend(['$q$: inventory','$p$: productivity'])

p_mean = np.mean(p_model)
p_std = np.std(p_model)

print(f"Movements mean: {p_mean}, std: {p_std}")

q_mean = np.mean(q_model)
q_std = np.std(q_model)

print(f"Inventory mean: {q_mean}, std: {q_std}")